In [ ]:
import pandas as pd
import os


# Google Drive file IDs extracted
microsoft_ads_file_id = '1S3Umb85XWFMxKmGvEM8k9PpcWP-dRit2'
google_ads_file_id = '1hp4saCRnhCOYK_VJHrFaFMlK8SvUX483'
meta_ads_file_id = '1VthFjrrgmyXbVzuy6e5soW9cus8DRw94'
website_landings_file_id = '1J8ghp2dOApvgH2IB1VUJwwDf81PMXccG'

#URL
microsoft_ads_url = f'https://drive.google.com/uc?export=download&id={microsoft_ads_file_id}'
google_ads_url = f'https://drive.google.com/uc?export=download&id={google_ads_file_id}'
meta_ads_url = f'https://drive.google.com/uc?export=download&id={meta_ads_file_id}'
website_landings_url = f'https://drive.google.com/uc?export=download&id={website_landings_file_id}'

#Load
microsoft_ads_df = pd.read_csv(microsoft_ads_url)
google_ads_df = pd.read_csv(google_ads_url)
meta_ads_df = pd.read_csv(meta_ads_url)
website_landings_df = pd.read_csv(website_landings_url)

# Preview the data
print("Microsoft Ads Data:")
print(microsoft_ads_df.head())
print("\nGoogle Ads Data:")
print(google_ads_df.head())
print("\nMeta Ads Data:")
print(meta_ads_df.head())
print("\nWebsite Landings Data:")
print(website_landings_df.head())

Microsoft Ads Data:
         Date     Campaign type  Impressions  Clicks   Cost  Conversions  \
0  2024-01-01          Audience       9132.0    50.0   26.8          0.0   
1  2024-01-01   Performance max        897.0     9.0    7.0          0.0   
2  2024-01-01  Search & content      95977.0   561.0  846.5          1.0   
3  2024-01-01          Shopping      59860.0   343.0  215.2          1.0   
4  2024-01-02          Audience       9926.0    49.0   25.9          0.0   

   Revenue  
0      0.0  
1      0.0  
2    168.8  
3    143.5  
4      0.0  

Google Ads Data:
         Date    Campaign type  Impressions  Clicks    Cost  Conversions  \
0  2024-01-01    Cross-network     143669.0   896.0   656.3          6.5   
1  2024-01-01  Display Network          3.0     0.0     0.0          0.0   
2  2024-01-01   Search Network       3701.0   251.0   496.5          4.5   
3  2024-01-01          YouTube      36211.0     8.0   115.2          0.0   
4  2024-01-02    Cross-network     183496.0  11

In [ ]:
microsoft_ads_df.fillna(0, inplace=True)
google_ads_df.fillna(0, inplace=True)
meta_ads_df.fillna(0, inplace=True)
website_landings_df.fillna(0, inplace=True)

for df in [microsoft_ads_df, google_ads_df, meta_ads_df, website_landings_df]:
    if 'date' in df.columns:
        df['date'] = pd.to_datetime(df['date'])

combined_data = pd.concat([microsoft_ads_df, google_ads_df, meta_ads_df], axis=0, ignore_index=True)


In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error

features = combined_data[['Cost', 'Impressions', 'Clicks']]
target = combined_data['Conversions']

X_train, X_test, y_train, y_test = train_test_split(features, target, test_size=0.2, random_state=42)

model = RandomForestRegressor(n_estimators=100, random_state=42)
model.fit(X_train, y_train)

y_pred = model.predict(X_test)
rmse = mean_squared_error(y_test, y_pred, squared=False)
print(f"Root Mean Squared Error: {rmse}")


Root Mean Squared Error: 6.888842365434301


In [ ]:
total_budget = 200000
min_allocation = total_budget * 0.10

microsoft_ads_df['Channel'] = 'Microsoft'
google_ads_df['Channel'] = 'Google'
meta_ads_df['Channel'] = 'Meta'

combined_data = pd.concat([microsoft_ads_df, google_ads_df, meta_ads_df], axis=0, ignore_index=True)


channel_performance = combined_data.groupby('Channel').agg({
    'Cost': 'sum',
    'Revenue': 'sum',
    'Conversions': 'sum'
})

channel_performance['budget_allocation'] = (channel_performance['Conversions'] / channel_performance['Conversions'].sum()) * total_budget

channel_performance['budget_allocation'] = channel_performance['budget_allocation'].apply(lambda x: max(x, min_allocation))

print(channel_performance[['budget_allocation']])


           budget_allocation
Channel                     
Google         139785.226294
Meta            27998.717769
Microsoft       32216.055937


In [ ]:
channel_output = channel_performance[['budget_allocation', 'Conversions']].reset_index()

channel_output.to_csv('media_investment_plan_output.csv', index=False)

print("Final Budget Allocation and Estimated Conversions:")
print(channel_output)


Final Budget Allocation and Estimated Conversions:
     Channel  budget_allocation  Conversions
0     Google      139785.226294      13954.2
1       Meta       27998.717769       2795.0
2  Microsoft       32216.055937       3216.0
